In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
result = load_dotenv(find_dotenv()) # read local .env file

print (os.environ["OPENAI_API_BASE"])

## Base Invocation

Base invocation allows you to simply call the LLM without worrying about anyting. Just place your prompt and go on.

In [ ]:
from langchain import PromptTemplate
from langchain_openai  import AzureChatOpenAI
from langchain import LLMChain

# Remember that the deployment name can be different from the model name
# and that you need to create a .env with the OPENAI_API_KEY variable
#and OPENAI_API_BASE
llm = AzureChatOpenAI(
    openai_api_version="2023-09-01-preview",
    deployment_name="GPT4o", #Deployment name
    azure_endpoint=os.environ["OPENAI_API_BASE"],
    model_name="GPT-4o"
)

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm
)

# user question
question = "Who is leonardo da vinci? Answer like capitan barbossa."
result = llm_chain.run(question)
print(result)


# Batch capabilities

If you need to have multiple prompts, you can simply pass a list of dictionaries to the `generate` method.


In [22]:
from pprint import pprint
qs = [
    {'question': "Who is leonardo da vinci? Answer like capitan barbossa."},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
]
batch_result = llm_chain.generate(qs)

# Loop through the results and print each question and its corresponding answer
for i, res in enumerate(batch_result.generations):
    print(f"Question {i+1}: {qs[i]['question']}")
    print(f"Answer {i+1}: {res[0].text}")
    print("\n")


Question 1: Who is leonardo da vinci? Answer like capitan barbossa.
Answer 1: Arrr, Leonardo da Vinci be a legendary figure, matey! He be an Italian polymath from the Renaissance, known fer his masterful skills in paintin', sculptin', engineerin', and inventin'. Why, his works like the Mona Lisa and The Last Supper be treasures of the art world, and his notebooks be brimming with fantastical designs and discoveries. A true genius of his time, he be!


Question 2: If I am 6 ft 4 inches, how tall am I in centimeters?
Answer 2: To convert a height of 6 feet 4 inches to centimeters, you can follow these steps:

1. Convert feet to inches: 6 feet × 12 inches/foot = 72 inches
2. Add the extra inches: 72 inches + 4 inches = 76 inches
3. Convert inches to centimeters: 76 inches × 2.54 cm/inch = 193.04 cm

So, if you are 6 feet 4 inches tall, you are approximately 193.04 centimeters tall.


Question 3: Who was the 12th person on the moon?
Answer 3: The 12th person to walk on the Moon was Harriso

a:\Develop\github\ai-playground\src\python\various\various\Lib\site-packages\langchain_openai\chat_models\base.py:369: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [30]:
# Print all properties of the first result to show its shape, do a pretty print
first_result = batch_result.generations[0]
first_answer = first_result[0]
print(f"The type of batch_result is: {type(batch_result)}")
print(f"The type of batch_result.generations is: {type(batch_result.generations)}")
print(f"The type of first_result is: {type(first_result)}")
print(f"The type of first_answer is: {type(first_answer)}")

pprint(vars(first_answer))


The type of batch_result is: <class 'langchain_core.outputs.llm_result.LLMResult'>
The type of batch_result.generations is: <class 'list'>
The type of first_result is: <class 'list'>
The type of first_answer is: <class 'langchain_core.outputs.chat_generation.ChatGeneration'>
{'generation_info': {'content_filter_results': {'hate': {'filtered': False,
                                                         'severity': 'safe'},
                                                'protected_material_code': {'detected': False,
                                                                            'filtered': False},
                                                'protected_material_text': {'detected': False,
                                                                            'filtered': False},
                                                'self_harm': {'filtered': False,
                                                              'severity': 'safe'},
                        

In [34]:
pprint(vars(first_answer.message))

{'additional_kwargs': {'refusal': None},
 'content': 'Arrr, Leonardo da Vinci be a legendary figure, matey! He be an '
            'Italian polymath from the Renaissance, known fer his masterful '
            "skills in paintin', sculptin', engineerin', and inventin'. Why, "
            'his works like the Mona Lisa and The Last Supper be treasures of '
            'the art world, and his notebooks be brimming with fantastical '
            'designs and discoveries. A true genius of his time, he be!',
 'example': False,
 'id': 'run-0343274c-0fd1-42d1-9607-4ab0d358f98a-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'content_filter_results': {'hate': {'filtered': False,
                                                           'severity': 'safe'},
                                                  'protected_material_code': {'detected': False,
                                                                              'filtered': False},
                          

In [43]:
response_metadata = first_answer.message.response_metadata

print("Content Filter Results:")
print(f"  Hate: Filtered: {response_metadata['content_filter_results']['hate']['filtered']}, Severity: {response_metadata['content_filter_results']['hate']['severity']}")
print(f"  Protected Material Code: Detected: {response_metadata['content_filter_results']['protected_material_code']['detected']}, Filtered: {response_metadata['content_filter_results']['protected_material_code']['filtered']}")
print(f"  Protected Material Text: Detected: {response_metadata['content_filter_results']['protected_material_text']['detected']}, Filtered: {response_metadata['content_filter_results']['protected_material_text']['filtered']}")
print(f"  Self Harm: Filtered: {response_metadata['content_filter_results']['self_harm']['filtered']}, Severity: {response_metadata['content_filter_results']['self_harm']['severity']}")
print(f"  Sexual: Filtered: {response_metadata['content_filter_results']['sexual']['filtered']}, Severity: {response_metadata['content_filter_results']['sexual']['severity']}")
print(f"  Violence: Filtered: {response_metadata['content_filter_results']['violence']['filtered']}, Severity: {response_metadata['content_filter_results']['violence']['severity']}")

print(f"Finish Reason: {response_metadata['finish_reason']}")
print(f"Logprobs: {response_metadata['logprobs']}")
print(f"Model Name: {response_metadata['model_name']}")

print("Prompt Filter Results:")
for prompt_filter in response_metadata['prompt_filter_results']:
    print(f"  Prompt Index: {prompt_filter['prompt_index']}")
    print(f"  Hate: Filtered: {prompt_filter['content_filter_results']['hate']['filtered']}, Severity: {prompt_filter['content_filter_results']['hate']['severity']}")
    print(f"  Jailbreak: Detected: {prompt_filter['content_filter_results']['jailbreak']['detected']}, Filtered: {prompt_filter['content_filter_results']['jailbreak']['filtered']}")
    print(f"  Self Harm: Filtered: {prompt_filter['content_filter_results']['self_harm']['filtered']}, Severity: {prompt_filter['content_filter_results']['self_harm']['severity']}")
    print(f"  Sexual: Filtered: {prompt_filter['content_filter_results']['sexual']['filtered']}, Severity: {prompt_filter['content_filter_results']['sexual']['severity']}")
    print(f"  Violence: Filtered: {prompt_filter['content_filter_results']['violence']['filtered']}, Severity: {prompt_filter['content_filter_results']['violence']['severity']}")

print(f"System Fingerprint: {response_metadata['system_fingerprint']}")

print("Token Usage:")
print(f"  Completion Tokens: {response_metadata['token_usage']['completion_tokens']}")
print(f"  Completion Tokens Details: {response_metadata['token_usage']['completion_tokens_details']}")
print(f"  Prompt Tokens: {response_metadata['token_usage']['prompt_tokens']}")
print(f"  Prompt Tokens Details: {response_metadata['token_usage']['prompt_tokens_details']}")
print(f"  Total Tokens: {response_metadata['token_usage']['total_tokens']}")

Content Filter Results:
  Hate: Filtered: False, Severity: safe
  Protected Material Code: Detected: False, Filtered: False
  Protected Material Text: Detected: False, Filtered: False
  Self Harm: Filtered: False, Severity: safe
  Sexual: Filtered: False, Severity: safe
  Violence: Filtered: False, Severity: safe
Finish Reason: stop
Logprobs: None
Model Name: gpt-4o-2024-05-13
Prompt Filter Results:
  Prompt Index: 0
  Hate: Filtered: False, Severity: safe
  Jailbreak: Detected: False, Filtered: False
  Self Harm: Filtered: False, Severity: safe
  Sexual: Filtered: False, Severity: safe
  Violence: Filtered: False, Severity: safe
System Fingerprint: fp_67802d9a6d
Token Usage:
  Completion Tokens: 87
  Completion Tokens Details: None
  Prompt Tokens: 27
  Prompt Tokens Details: None
  Total Tokens: 114


You can instead simply create a single prompt, but ask the LLM to answer all the questions one at a time


In [48]:
multi_template = """Answer the following questions one at a time.
Separate each new answer with ----

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm
)

qs_str = (
    "Who is leonardo da vinci? Answer like capitan barbossa.\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" 
)

pprint(llm_chain.run(qs_str))


('Ah, Leonardo da Vinci, ye say? Aye, he be a legendary figure from the '
 "Renaissance, a master of many trades, includin' paintin', inventin', and "
 'anatomy. His works be timeless treasures, like the Mona Lisa and The Last '
 'Supper, and he be a man of boundless curiosity and genius, savvy?\n'
 '\n'
 '----\n'
 '\n'
 'If ye be 6 feet 4 inches tall, ye stand at a mighty 193.04 centimeters. To '
 'be precise, ye take the feet, multiply by 30.48, and add the inches '
 'multiplied by 2.54. So, 6 times 30.48 plus 4 times 2.54 gives ye 193.04 cm.\n'
 '\n'
 '----\n'
 '\n'
 'The 12th soul to set foot upon the lunar surface be none other than Harrison '
 "Schmitt. He walked the moon durin' the Apollo 17 mission in December 1972, "
 'the last of the Apollo missions to land on that distant orb.')


In [53]:

complex_result = llm_chain.generate([{"questions": qs_str}])
res = complex_result.generations[0][0]
response_metadata = res.message.response_metadata 

print (res.message.content)

print('\n')
print("Token Usage:")
print(f"  Completion Tokens: {response_metadata['token_usage']['completion_tokens']}")
print(f"  Completion Tokens Details: {response_metadata['token_usage']['completion_tokens_details']}")
print(f"  Prompt Tokens: {response_metadata['token_usage']['prompt_tokens']}")
print(f"  Prompt Tokens Details: {response_metadata['token_usage']['prompt_tokens_details']}")
print(f"  Total Tokens: {response_metadata['token_usage']['total_tokens']}")


Ah, ye be askin' about Leonardo da Vinci, eh? Aye, he be a legendary figure from the days of old, a true Renaissance man. He painted, he sculpted, he invented, and he pondered the mysteries of the world. A master of many trades, he was, just like a cunning pirate who knows his sea charts and treasure maps by heart!

----
If ye stand tall at 6 feet 4 inches, then in the ways of the metric system, ye be about 193 centimeters. 

----
As for the 12th soul to set foot upon the lunar surface, that be Harrison Schmitt, a geologist and astronaut of Apollo 17. He walked the moon's dusty plains in December of 1972, the last of his kind to do so.



Token Usage:
  Completion Tokens: 163
  Completion Tokens Details: None
  Prompt Tokens: 70
  Prompt Tokens Details: None
  Total Tokens: 233
